In [1]:
! pip install -q tensorflow tensorflow-hub 

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

def load_yamnet_model():
    # Path to the YAMNet .pb file
    model_fn = './saved_model.pb'
    
    # Create a new graph
    graph = tf.Graph()
    with graph.as_default():
        # Create a session within this graph
        sess = tf.InteractiveSession(graph=graph)
        
        # Load the model from the .pb file
        with tf.io.gfile.GFile(model_fn, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
        
        # Define the input tensor for the audio waveform
        t_input = tf.placeholder(dtype=tf.float32, shape=[None], name='waveform')
        
        # Import the graph definition into this graph
        tf.import_graph_def(graph_def, name='', input_map={'waveform': t_input})
    
    # Output tensors from the graph
    scores_tensor = graph.get_tensor_by_name('scores:0')  # Per-frame predicted scores
    embeddings_tensor = graph.get_tensor_by_name('embeddings:0')  # Embeddings
    log_mel_spectrogram_tensor = graph.get_tensor_by_name('log_mel_spectrogram:0')  # Log Mel Spectrogram
    
    return graph, sess, t_input, scores_tensor, embeddings_tensor, log_mel_spectrogram_tensor


In [4]:
# Load the YAMNet model
graph, sess, t_input, scores_tensor, embeddings_tensor, log_mel_spectrogram_tensor = load_yamnet_model()

# Example: Dummy waveform input
dummy_waveform = np.random.uniform(-1.0, 1.0, size=(16000,))  # 1 second of random audio at 16 kHz

# Run the model
scores, embeddings, log_mel_spectrogram = sess.run(
    [scores_tensor, embeddings_tensor, log_mel_spectrogram_tensor],
    feed_dict={t_input: dummy_waveform}
)

print("Scores Shape:", scores.shape)
print("Embeddings Shape:", embeddings.shape)
print("Log Mel Spectrogram Shape:", log_mel_spectrogram.shape)


ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


NotFoundError: NewRandomAccessFile failed to Create/Open: ./saved_model.pb : The system cannot find the file specified.
; No such file or directory